In [ ]:
import googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
autenticação = ['https://www.googleapis.com/auth/analytics.readonly']
dados = 'Endereço do arquivo.json'
usuario_id = " ***** "
data_inicio = '2020-09-25'
data_final = '2020-10-26'

In [ ]:
#essa funçao vai validar o login do Analytics e a API liberar o acesso
def initialize_analyticsreporting():
    credentials = Credentials.from_service_account_file(dados, fonte=FONTE)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

In [ ]:
#função extrai os dados que queremos seguindo o conceito
def get_report_visao_geral(analytics):
    return analytics.reports().batchGet(body={'reportRequests': [{
        'viewId': VIEW_ID
        'dateRanges': [{'startDate': data_inicio, 'endDate': data_final}],
        'dimensions': [
            {'name': 'ga:date'}
        ], 
          'metrics': [
            {'expression': 'ga:totals.pageviews'},
            {'expression': 'ga:sessions'},
            {'expression': 'ga:totals.timeOnScreen'},
            {'expression': 'ga:transactions'},
            {'expression': 'ga:device.browser'},
        ]  
    }]}).execute()

In [ ]:
def response(response):
    report = response.get('reports', [])[0] 
    header_dimensions = report.get('columnHeader', {}).get('dimensions', [])
    header_metrics = [value['name'] for value in report.get('columnHeader', {}).get('metricHeader', {}).get('metricHeaderEntries', [])]
    headers = header_dimensions + header_metrics
    headers = list(map((lambda x: x.split(':', 1)[-1]), headers))
    values = []
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        values_dimensions = row.get('dimensions', [])
        values_metrics = row.get('metrics', [])[0].get('values', [])
        values.append(values_dimensions + values_metrics)
    df = pd.DataFrame(columns=headers, data=values)
    return df

In [ ]:
analytics = initialize_analyticsreporting()
response_visao_geral = get_report_visao_geral(analytics)
df = response(response_visao_geral)

In [ ]:
df.head()